In [ ]:
from mmengine.config import Config
from mmocr.registry import DATASETS
import sys
import os
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)
from lmdb_dataset import CustomLMDBDataset # Import your custom dataset class

# Load config
cfg = Config.fromfile('../configs/etv_test1.py')

# Build dataset
dataset = DATASETS.build(cfg.train_dataloader.dataset)

# Kiểm tra 1 sample đầu tiên
sample = dataset[0]
print(sample)


In [ ]:
img, instances = sample['img'], sample['instances']
import matplotlib.pyplot as plt

plt.imshow(img)
plt.axis('off')
plt.show()